In [ ]:
import sagemaker
import boto3
import pandas as pd

from sagemaker import image_uris
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from time import gmtime, strftime
import time

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = ''
prefix = ''

role = sagemaker.get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)

## Select container
https://docs.aws.amazon.com/sagemaker/latest/dg/ecr-us-east-2.html

In [ ]:
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

## Set location for training and validation data

IMPORTANT - XGBoost REQUIRES target column to be first, and there can be NO HEADER column row

In [ ]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train.csv', content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate.csv', content_type='csv')

## Set up XGBoost algorithm
https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html

https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/using_xgboost.html#create-an-estimator

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output/xgboost'.format(bucket,prefix),
                                    sagemaker_session=sess,
                                    enable_sagemaker_metrics=True)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        eval_metric='auc',
                        num_round=100)
 

In [ ]:
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

## HPO
https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html

In [ ]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}

objective_metric_name = 'validation:auc'

tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=20,
                            max_parallel_jobs=3)

In [ ]:
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

In [ ]:
boto3.client('sagemaker').describe_training_job(TrainingJobName=tuner.best_training_job())